# PPR 

<img src="{{site.baseurl}}/images/ppr4r.png" width = "100%">



# List being Created:

    async function loadAttempts() {
    const currentUserResponse = await fetch(${pythonURI}/api/id, fetchOptions);
    if (!currentUserResponse.ok) throw new Error('Failed to fetch current user');
    const currentUser = await currentUserResponse.json();
    userName = currentUser.uid;
    userID = currentUser.id;

    const quizGrading = await fetch(quizGradingsApi, fetchOptions);
    if (!quizGrading.ok) {console.error("Error loading attempts:", quizGrading);}

    const quizResults = await quizGrading.json();
    console.log(quizResults);
    }

**Explanation:** The list quizResults is created by fetching the quiz attempts and scores from the backend and storing it in a variable to be processed and then displayed and loaded on the table. 

# List being processed:

    quizResults.forEach(attempt => {
    const row = document.createElement('tr');
    const idCell = document.createElement('td');
    idCell.innerHTML = attempt.id;
    const usernameCell = document.createElement('td');
    usernameCell.innerHTML = attempt.username;
    const quizgradeCell = document.createElement('td');
    quizgradeCell.innerHTML = attempt.quizgrade;
    const attemptCell = document.createElement('td');
    attemptCell.innerHTML = attempt.attempt;
    const actionCell = document.createElement('td');
    
    if (attempt.username === userName) {
        const deleteButton = document.createElement('button');
        deleteButton.innerHTML = 'Delete';
        deleteButton.addEventListener('click', () => deleteAttempt(attempt.id));
        const editButton = document.createElement('button');
        editButton.innerHTML = 'Edit';
        editButton.addEventListener('click', () => editAttempt(attempt.id));
        actionCell.append(deleteButton);
        actionCell.append(editButton);
    }
    
    row.append(idCell);
    row.append(usernameCell);
    row.append(quizgradeCell);
    row.append(attemptCell);
    row.append(actionCell);
    tbody.append(row);
    });

**Explanation:** After fetching the quizResults, it's processed by iterating through the list and updating the DOM (displaying the attempts in the table). The quizResults array is processed by iterating over each attempt, creating a new table row for each one, and then displaying the data inside the row.

# A Function:

    async function loadAttempts() {
    }

**Explanation:** The function responsible for loading and processing the quiz attempt data is loadAttempts. It fetches data and populates the table with quiz scores and attempts.

# Call to Function:

    window.onload = () => {
    const selectedQuestions = randomizeQuestions(Questions, 5);
    buildQuiz(selectedQuestions);
    loadAttempts(); 
    };

**Explanation:** The loadAttempts function is called and ensures that the quiz attempts and scores are displayed in the table after the page reloads.


# CPT table requirements:

| Requirement                | Notification Feature                                      |
|----------------------------|----------------------------------------------------------|
| Input                      | Starts off by user actions like submiting the quiz  |
| Use of List | Stored in an SQLite database.          |
| Procedure                  | Quiz attempt is made, linked to users, and saved.       |
| Algorithm                  | Quiz submissions are made, attempt and score is added for a user.       |
| Output                     | users see quiz attempt and scores with details.               |
| Functionality Demonstration | User action creates a quiz attempt and score, which appear on the site. |


# Input:

    document.getElementById('submit').addEventListener('click', () => {
    showResults(selectedQuestions);
    });
 
Explanation: This event listener detects when the user submits the quiz. Calls showResults(selectedQuestions), which processes the answers.

# Use of List:

    class QuizAttempt(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'))
    username = db.Column(db.String(100))
    quizgrade = db.Column(db.Integer)
    attempt_date = db.Column(db.DateTime, default=datetime.utcnow)


Explanation: This stores quiz attempts in SQLite. The user_id links the attempt to the correct user.

# Procedure:

    const attemptData = {
    quizgrade: numCorrect,
    attempt: new Date().toISOString(),
    id: userID,
    username: userName,
    };

    fetch(quizGradingsApi, {
    method: "POST",
    headers: { "Content-Type": "application/json" },
    body: JSON.stringify(attemptData),
    })
    .then(response => response.json())
    .then(data => {
        console.log("Score stored successfully:", data);
        loadAttempts(); 
    })
    .catch(error => console.error("Error storing score:", error));

Explanation:Sends The quiz attempt data (score, user ID, date) to the backend. Backend saves it in the database. Calls loadAttempts() to update the UI with the new attempt.

# Algorithm: 

    async function showResults(questions) {
    let numCorrect = 0;
    questions.forEach((currentQuestion, questionNumber) => {
        const answerContainer = document.querySelector(`input[name=question${questionNumber}]:checked`);
        const userAnswer = answerContainer ? answerContainer.value : null;

        if (userAnswer === currentQuestion.correctAnswer) {
            numCorrect++;
        }
    });

    document.getElementById('results').innerHTML = `${numCorrect} out of ${questions.length}`;
    
    const attemptData = {
        quizgrade: numCorrect,
        attempt: new Date().toISOString(),
        id: userID,
        username: userName,
    };

    fetch(quizGradingsApi, {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify(attemptData)
    })
    .then(response => response.json())
    .then(() => loadAttempts()); 
    }   

Explanation: Iterates through answers and counts correct ones. Sends score and user details to backend. Calls loadAttempts() to update the UI.

# Output:

    async function loadAttempts() {
    const quizGrading = await fetch(quizGradingsApi, fetchOptions);
    if (!quizGrading.ok) {
        console.error("Error loading attempts:", quizGrading);
    }

    const quizResults = await quizGrading.json();
    const table = document.getElementById('attemptsTable').querySelector('tbody');
    table.innerHTML = '';

    quizResults.forEach(attempt => {
        const row = document.createElement('tr');
        row.innerHTML = `
            <td>${attempt.id}</td>
            <td>${attempt.username}</td>
            <td>${attempt.quizgrade}</td>
            <td>${attempt.attempt}</td>
            <td>
                ${attempt.username === userName ? `<button onclick="deleteAttempt(${attempt.id})">Delete</button>` : ''}
            </td>
        `;
        table.appendChild(row);
    });
    }

Explanation: Fetches all quiz attempts from the backend. Populates the table dynamically. Only allows deletion of user’s own attempts.